# Amazon SageMaker 模型偏差监控：利用 SageMaker Clarify 实现公平性和可解释性

Amazon SageMaker Clarify 可以检测潜在偏差和帮助解释这些模型的预测方式，从而有助于改进机器学习模型。SageMaker Clarify 提供的公平性和可解释性功能进一步使 AWS 客户能够构建可靠且可理解的机器学习模型。该产品附带的工具可以帮助您完成以下任务。

– 衡量在机器学习 (ML) 生命周期的每个阶段（数据收集、模型训练和优化以及监控部署的用于推理的机器学习模型）可能存在的偏差。
– 生成针对风险与合规性团队以及外部监管机构的模型监管报告。
– 解释数据、模型以及用于评估预测的监控措施。

## 导入

In [ ]:
import os
import copy
import json
import random
import time
import datetime
import boto3
import pandas as pd

from datetime import datetime, timedelta

from sagemaker import get_execution_role, image_uris, Session
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.predictor import Predictor

from sagemaker.model_monitor import (
    BiasAnalysisConfig,
    CronExpressionGenerator,
    DataCaptureConfig,
    EndpointInput,
    ExplainabilityAnalysisConfig,
    ModelBiasMonitor,
    ModelExplainabilityMonitor,
)

from sagemaker.clarify import (
    BiasConfig,
    DataConfig,
    ModelConfig,
    ModelPredictedLabelConfig,
    SHAPConfig,
)

### 此任务的配置

In [ ]:
# Replace below <bucket_name> with the value the modelArtifactBucket value at the left of the instruction page
bucket = '<bucket_name>'

# Get production endpoint name
endpoint_name = boto3.Session().client('sagemaker').list_endpoints(SortBy='CreationTime')['Endpoints'][0]['EndpointName']

# Get production model name
model_name = boto3.Session().client('sagemaker').list_models(SortBy='CreationTime')['Models'][0]['ModelName']


print(f"Bucket Name :{bucket}")
print(f"Production Endpoint Name:{endpoint_name}")
print(f"Production Model Name:{model_name}")

In [ ]:
role = get_execution_role()
print(f"RoleArn: {role}")

sagemaker_session = Session()
sagemaker_client = sagemaker_session.sagemaker_client
sagemaker_runtime_client = sagemaker_session.sagemaker_runtime_client

region = sagemaker_session.boto_region_name
print(f"AWS region: {region}")

In [ ]:
print(f"Demo Bucket: {bucket}")
prefix = f'sagemaker/DEMO-ClarifyModelMonitor-{datetime.now():%Y%m%d%H%M%S}'
s3_key = f"s3://{bucket}/{prefix}"
print(f"S3 key: {s3_key}")

code_prefix = '{}/code'.format(prefix)
data_capture_prefix = f'{prefix}/datacapture'
s3_capture_upload_path = f'{s3_key}/datacapture'
s3_report_path = f'{s3_key}/reports'

print(f"Capture path: {s3_capture_upload_path}")
print(f"Report path: {s3_report_path}")

endpoint_instance_count = 1
endpoint_instance_type = "ml.m5.xlarge"

### 数据文件

读取数据文件

In [ ]:
test_dataset = "data/test.csv"
validation_dataset = "data/train.csv"
dataset_type = "text/csv"

with open(validation_dataset) as f:
    headers_line = f.readline().rstrip()
all_headers = headers_line.split(",")
label_header = all_headers[0]

### 在端点上启用数据捕获

要为监控任务启用数据捕获，请指定名为 DataCaptureConfig 的新捕获选项，它支持捕获端点的请求负载和响应负载。此捕获配置适用于所有变体。

此操作通常需要 10 到 15 分钟。

**注意**：这可以通过 CloudFormation 自动执行（审核员：请提供建议）

In [ ]:
data_capture_config = DataCaptureConfig(
                        enable_capture = True,
                        sampling_percentage=100,
                        destination_s3_uri=s3_capture_upload_path,
                        kms_key_id=None,
                        capture_options=["REQUEST", "RESPONSE"],
                        csv_content_types=["text/csv"],
                        json_content_types=["application/json"])



# Now it is time to apply the new configuration and wait for it to be applied
predictor = Predictor(endpoint_name=endpoint_name)
predictor.update_data_capture_config(data_capture_config=data_capture_config)
sagemaker_session.wait_for_endpoint(endpoint=endpoint_name)

### 通过样本流量测试端点

In [ ]:
# Here we are randomly picking subset of data from test datasets.
import itertools

shape = pd.read_csv(test_dataset)

a = [10*i for i in range(3)]
b = [10+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]

test_data = shape.drop(shape.columns[[0]],axis=1)
test_data = test_data.iloc[indices]
test_data_with_label = shape.iloc[indices]

In [ ]:
test_data.to_csv("data/data-test.csv",index=False,header=False)
test_data_with_label.to_csv("data/data-test-label.csv",index=False,header=False)

In [ ]:
# Invoke Endpoint with test data for a quick test
print(f"Sending test traffic to the endpoint {endpoint_name}. \nPlease wait...")
predictions = ''

with open('data/data-test.csv', 'r') as f:
    for row in f:
        print(".", end="", flush=True)
        payload = row.rstrip('\n')
        response = sagemaker_runtime_client.invoke_endpoint(EndpointName=endpoint_name,
                                   ContentType="text/csv",
                                   Body=payload)
        predictions = ','.join([predictions, response['Body'].read().decode('utf-8')])
        time.sleep(0.5)

predictions = predictions.replace('\n','')
predictions = predictions.split(",")
predictions.pop(0)
print("="*20)
print(predictions)
print("Done!") 

## 查看捕获的数据

现在，列出存储在 Amazon S3 中的数据捕获文件。您将看到来自不同时间段的不同文件，这些时间段基于调用发生的时间进行组织。Amazon S3 路径的格式如下所示：

`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [ ]:
s3_client = boto3.Session().client('s3')
current_endpoint_capture_prefix = '{}/{}'.format(data_capture_prefix, endpoint_name)
result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
print(result)
str_error = "waiting for the file to load..."
while str_error:
    try:
        capture_files = [capture_file.get("Key") for capture_file in result.get('Contents')]
        print(capture_files)
        str_error=None
    except:
        print(str_error)
        time.sleep(10)
        contine
print("Found Capture Files:")
print("\n ".join(capture_files))

接下来，查看单个捕获文件的内容。此时，您将看到 Amazon SageMaker 特定 JSON-line 格式的文件中捕获的所有数据。快速查看捕获的文件中的前几行。

In [ ]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get('Body').read().decode("utf-8")

capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

最后，单行内容显示在格式化 JSON 文件中的下方，以便您可以更好地查看。

In [ ]:
import json
print(json.dumps(json.loads(capture_file.split('\n')[0]), indent=2))

### Amazon Sagemaker Clarify 设置

**注意**：以下代码示例摘自 Amazon SageMaker 提供的 [示例代码](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker_processing/fairness_and_explainability/fairness_and_explainability.ipynb)。其中一些代码已根据实验要求进行更新。

In [ ]:
# Run SageMaker Clarify processor using the following command
from sagemaker import clarify
clarify_processor = clarify.SageMakerClarifyProcessor(role=role,
                                                      instance_count=1,
                                                      instance_type='ml.m5.xlarge',
                                                      sagemaker_session=sagemaker_session)

将测试数据集和训练数据集上载到 S3。_train_uri_ 和 _test_uri_ 将用于在后续步骤中直接从 S3 写入 DataConfig 和 ModelConfig 中提取数据。

In [ ]:
from sagemaker.s3 import S3Uploader
from sagemaker.inputs import TrainingInput

train_uri = S3Uploader.upload('data/train.csv', 's3://{}/{}'.format(bucket, prefix))
train_input = TrainingInput(train_uri, content_type='csv')
test_uri = S3Uploader.upload('data/test.csv', 's3://{}/{}'.format(bucket, prefix))

### 检测偏差

SageMaker Clarify 可以帮助您利用各种指标检测训练前和训练后可能存在的偏差。

### 写入 DataConfig 和 ModelConfig

**DataConfig** 对象向 SageMaker Clarify 传达一些有关数据输入/输出的基本信息。您将指定在何处查找输入数据集、在何处存储输出、目标列 (label)、标头名称和数据集类型。

In [ ]:
bias_report_output_path = 's3://{}/{}/clarify-bias'.format(bucket, prefix)
bias_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                      s3_output_path=bias_report_output_path,
                                      label='Churn',
                                      headers=all_headers,
                                      dataset_type='text/csv')

**ModelConfig** 对象传达有关经过训练的模型的信息。为了避免额外流量进入生产模型，SageMaker Clarify 会在处理时设置和删除专用端点。

– **instance_type** 和 **instance_count** 指定用于在 SageMaker Clarify 处理期间运行模型的首选实例类型和实例计数。测试数据集较小，因此单个标准实例足以运行此示例。如果您的数据集大而复杂，可能需要使用更好的实例类型来加快速度，或者添加更多实例来启用 Spark 并行化。

– **accept_type** 表示端点响应负载格式，而 **content_type** 表示向端点发送的请求的负载格式。

In [ ]:
model_config = clarify.ModelConfig(model_name=model_name,
                                   instance_type='ml.m5.xlarge',
                                   instance_count=1,
                                   accept_type='text/csv',
                                   content_type='text/csv')

**ModelPredictedLabelConfig** 提供有关预测格式的信息。随机森林模型输出样本的概率，因此 SageMaker Clarify 调用端点，然后使用 probability_threshold 将概率转换为二进制标签以便进行偏差分析。高于阈值的预测被解释为标签值 1，而低于或等于阈值的预测被解释为标签值 0。

In [ ]:
predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

### 写入 BiasConfig
SageMaker Clarify 还需要有关敏感列 (facets) 有哪些、敏感特征 (facet_values_or_threshold) 可能有哪些以及期望结果是什么 (label_values_or_threshold) 的信息。SageMaker Clarify 可以处理 facet_values_or_threshold 和 label_values_or_threshold 的分类数据和连续数据。在本示例中，我们将使用分类数据。

我们将在 BiasConfig API 中指定此信息。在本示例中，积极结果是 Churn = 0，_Account Length_ 是敏感分类，_Day Calls_ 响应者是敏感组。group_name 用于形成衡量指标的子组。

In [ ]:
bias_config = clarify.BiasConfig(label_values_or_threshold=[0],
                                facet_name='Account Length',
                                facet_values_or_threshold=[100],
                                group_name='Day Calls')

**训练前存在的偏差**

在进行任何模型训练之前，数据中就可能存在偏差。在训练开始前检查数据是否存在偏差，这可以帮助您发现任何数据收集漏洞、帮助您进行特征工程，以及帮助您了解数据可能反映的社会偏见。

计算训练前存在的偏差指标不需要使用经过训练的模型。

**训练后存在的偏差**

计算训练后存在的偏差指标不需要使用经过训练的模型。

无偏差的训练数据（由偏差指标衡量的公平性概念决定）在训练后仍然可能会导致有偏差的模型预测。这种情况是否会发生取决于多项因素，包括超参数的选择。

您可以使用 run_pre_training_bias() 和 run_post_training_bias() 分别运行这些选项，也可以如下所示使用 run_bias() 同时运行这些选项。

In [ ]:
clarify_processor.run_bias(data_config=bias_data_config,
                           bias_config=bias_config,
                           model_config=model_config,
                           model_predicted_label_config=predictions_config,
                           pre_training_methods='all',
                           post_training_methods='all')

**查看偏差报告**

您可以在 Studio 中的 <b>experiments</b>（实验）选项卡下查看结果。

<img src="./recordings/bias_report.gif">

每个偏差指标都有详细的解释，其中包含您可以探索的示例。

<img src="./recordings/bias_detail.gif">

您也可以在方便使用的表中总结结果！

<img src="./recordings/bias_report_chart.gif">



如果您还不是 Studio 用户，可以访问以下 S3 存储桶中 pdf、html 和 ipynb 格式的偏差报告：

In [ ]:
bias_report_output_path

### 解释预测

不断扩大的业务需求和法律法规要求解释为什么模型会做出这样的决定。SageMaker Clarify 使用 SHAP 来解释每个输入特征对最终决定的贡献。

Kernel SHAP 算法需要一个基准（也称为背景数据集）。基准数据集类型应与 DataConfig 的 dataset_type 相同，基准样本应仅包含特征。根据定义，基准应是基准数据集文件的 S3 URI，或者是一个适当的样本列表。在本示例中，我们选择了后者，并将测试数据集的第一个样本添加到了列表中。

In [ ]:
shap_config = clarify.SHAPConfig(baseline=[test_data.iloc[0].values.tolist()],
                                 num_samples=15,
                                 agg_method='mean_abs',
                                 save_local_shap_values=False)

explainability_output_path = 's3://{}/{}/clarify-explainability'.format(bucket, prefix)
explainability_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                s3_output_path=explainability_output_path,
                                label='Churn',
                                headers=all_headers,
                                dataset_type='text/csv')

In [ ]:
clarify_processor.run_explainability(data_config=explainability_data_config,
                                     model_config=model_config,
                                     explainability_config=shap_config)

#### 查看可解释性报告
与偏差报告一样，您可以在 Studio 中的 <b>experiments</b>（实验）选项卡下查看可解释性报告


<img src="./recordings/explainability_detail.gif">

<b>Model Insights</b>（模型洞察）选项卡包含直接指向报告和模型洞察的链接。

如果您还不是 Studio 用户，与偏差报告一样，您可以在以下 S3 存储桶中访问该报告。

In [ ]:
explainability_output_path

### 清除

最后，请不要忘记清除我们在本演示中设置和使用的资源！

In [ ]:
session.delete_model(model_name)